#

Analytic code snippets from The Epidemiology Research Group on Organ Transplantation

+ Look for code by Ctrl+F 
+ Be aware that there will be a 6-month lag in SRTR data update for outcomes that need follow-up, such as death and graft failure
+ So for example if you have Sep 2015 data set, it’s safer to include data in or earlier than Mar 2015 for analysis of these outcomes 
+ For candidate datasets, `can_endwlfu` indicates the last date of follow-up For donor dispositions, such as discard, there is no lag 
+ Discuss how to model missing data with your mentor- model-wise deletion? Imputation? <strike>Missing indicator</strike>? 

---

## 1. Patient survival/death

```stata
// -------------------------------------------
// Patient survival /death
// -------------------------------------------

*Date of mortality*
egen death_dt = rowmin( pers_ssa_death_dt pers_optn_death_dt pers_restrict_death_dt tfl_death_dt)
label var death_dt "Date of death"
****A time metric taking administrative censoring into consideration****
gen t1=death_dt
replace t1=mdy(03,04,2015) if death_dt>mdy(03,04,2015)  // modify according to study time frame
label var t1 "Time metric for Death"
****Mortality, only those happened before the end of F/U****
gen death=inrange(death_dt,rec_tx_dt,t1)
label var death "All-cause mortality"
label define death 0 "Alive" 1 "Dead"
label values death death

//DEATH DATE
//when using candidate data set only
egen death_dt=rowmin(pers_ssa_death_dt pers_optn_death_dt pers_restrict_death_dt tfl_death_dt)
replace death_dt=can_rem_dt if can_rem_dt<death_dt &
inlist(can_rem_cd,8)
//died (8)

*WAITLIST MORTALITY candidate data
egen death_dt=rowmin(pers_ssa_death_dt pers_optn_death_dt pers_restrict_death_dt tfl_death_dt)
replace death_dt=can_rem_dt if can_rem_dt<death_dt &
inlist(can_rem_cd,8,13,5)  //8-died; 13-deterioration; 5- medically unsuitable
replace death_dt=. if can_rem_cd<death_dt & inlist(can_rem_cd,4,15,21,14,22,18,19) //4-DDTx; 15-LDTx; 21
//censor if removal is for tx
//discuss with your mentor
```

---

## 2. Graft survival (death censored and all-cause)

```stata
// -------------------------------------------
// Graft survival / failure GF (death censored and all-cause)
// -------------------------------------------
gen t1=death_dt
replace t1=mdy(03,04,2015) if death_dt>mdy(03,04,2015)
//modify this based on how you define your study period!
****Date for all-cause graft failure****
//Some of these dates included in the brackets are version-specific. Check the SAF dataset you work on t
// rec_esrd_grf_fail_dt & rec_esrd_fail_dt are only in Oct 14 dataset
label var acgf_dt "Date of all-cause graft failure"
//MUST ascertain death_dt prior to estimating gfail_dt
// check that death dates are post transplant
*compare death_dt tx_dt
gen acgf=inrange(acgf_dt,rec_tx_dt,t1)
label var acgf "All-cause graft failure"
label define acgf 0 "Graft functioning up to administrative censor" 1 "Graft failed or died"
label values acgf acgf
****Graft failure date for Death-censored graft failure models****
****DCGF date****
// need to censor death later (still GF if gf_dt == death_dt
gen dcgf_dt = min(tfl_graft_dt, rec_esrd_fail_dt, rec_esrd_grf_fail_dt, pers_retx, rec_fail_dt, rec_resu
// rec_esrd_grf_fail_dt & rec_esrd_fail_dt are only in 1410 dataset
label var dcgf_dt "Date of DCGF"
gen dcgf=inrange(dcgf_dt,rec_tx_dt,min(death_dt, mdy(12,31,2015)))
//Change the date according to your study period
label var dcgf "Death-censored graft failure"
label define dcgf 0 "Graft functioning" 1 "Graft failed <=death date or administrative censor"
label values dcgf dcgf
```

---

## 3. Delayed Graft Function (DGF)

```stata
// -------------------------------------------
// ascertain DELAYED GRAFT FUNCTION
// -------------------------------------------
// Keep it in mind- if DGF is the main exposure/outcome, those with  dgf = 0 and died within the first w
// main analysis 1) adjust for pre-emptive as covariate, 2) exclude any pre-emptives (see mentor) - pre-
gen dgf=rec_first_week_dial=="Y"
replace dgf = . if rec_first_week_dial=="U" | missing(rec_first_week_dial)
```

---

## 4. Primary Non-Function (PNF)

```stata
// -------------------------------------------
// ascertain primary non-function
// -------------------------------------------
// this is PNF for patients who survive at least 90 days
// CAUTION: anyone who doesn't survive to 90 days, they are dropped
//90 days and 1 week shown- ask your mentor re: the appropriate measure
//determine gfail_dt appropriately (death censored, not all-cause)
//Option 1 works fine if you are just excluding PNF folks. If PNF is a covariate, then, what is now code
gen byte pnf = (gfail_dt - rec_tx_dt <= 90) // doesn't need to be dgf=="Y"
replace pnf = . if death_dt < gfail_dt & death_dt-rec_tx_dt <= 90

/*Defined as "the lack of kidney graft function recovery and
permanent need of dialysis since the first week of transplant surgery
(Jayaram, Clin Transplantation, 2012)*/
gen pnf=(gfail_dt-rec_tx_dt<=7)
replace pnf=. if gfail_dt==. | rec_tx_dt==.
label var pnf "Permanent non-functioning"
label define pnf 0 "No PNF" 1 "PNF"
label values pnf pnf
```

---


## 5. Acute rejection

```stata
// -----------------------------------
// 1-yr acute rejection
// -----------------------------------
1. Simple version (need to re-test)
use txf_ki.dta, clear
gen fu_time=tfl_px_stat_dt-rec_tx_dt
gen within1yr=(inlist(tfl_fol_cd,3,6,10)|inlist(tfl_fol_cd, 800,801, 998, 999)) & fu_time<366
// within1yr==1 if (1) the data was from 3, 6, 12 months post-KT or (2) death, gloss or ltfu records tha
keep if within1yr==1
gen ar_bin = cond(inrange(tfl_acute_rej_episode, 1, 2), 1, 0) if tfl_acute_rej_episode !=998 & !missing(
gen ar_biopsy=(tfl_acute_rej_biopsy_confirmed==2)
tab ar_bin ar_biopsy
gen ar_treat=.
replace ar_treat=1 if tfl_rej_treat=="Y"
replace ar_treat=0 if tfl_rej_treat=="N"
gen ar_bin_fail=.
replace ar_bin_fail=1 if tfl_fail_rej_acute=="Y"
replace ar_bin_fail=0 if tfl_fail_rej_acute=="N"
gen ar_any=max(ar_bin, ar_biopsy, ar_treat, ar_bin_fail)
collapse (max) ar_any* ar_biopsy (first) rec_tx_dt, by(trr_id)
** ar_biopsy is only if year(rec_tx_dt)>=2004 and may be if ar_any==0.
compress
tempfile ar
save `ar'
global srtrdir="/dcs01/igm/segevlab/data/srtr1509/public"
local rejfile="$srtrdir/txf_ki.dta"
use `rejfile', clear
keep if rec_tx_ty==1   //this code is only for DDKT

gen fu_time=tfl_px_stat_dt-rec_tx_dt
gen m6=((inlist(tfl_fol_cd,3,6)|(inlist(tfl_fol_cd, 800, 998, 999)) & fu_time<183))
gen within1yr=((inlist(tfl_fol_cd,3,6,10)|(inlist(tfl_fol_cd, 800,801, 998, 999)) & fu_time<366))
// within1yr==1 if (1) the data was from 3, 6, 12 months post-KT or (2) death, gloss or ltfu happened wi
gen m24=((inlist(tfl_fol_cd,3,6,10,20)|(inlist(tfl_fol_cd, 800, 801,998, 999)) & fu_time<731))
***Clinical***
rename tfl_acute_rej_episode ar
gen ar_bin= cond(inrange(ar, 1, 2), 1, 0) if ar !=998 & !missing(ar)
replace ar_bin=. if ar==998 | ar==.
***Biopsy***
gen ar_bx=tfl_acute_rej_biopsy_confirmed==2
replace ar_bx=0 if tfl_acute_rej_biopsy_confirmed==3
replace ar_bx=. if tfl_acute_rej_biopsy_confirmed==1 | tfl_acute_rej_biopsy_confirmed==998 | tfl_acute_r
gen ar_bx2=ar_bx
replace ar_bx2=0 if ar_bin==0 & ar_bx2==.
replace ar_bx2=1 if ar_bin==1 & ar_bx2==.
rename tfl_rej_treat ar_treat
replace ar_treat = "1" if ar_treat=="Y"
replace ar_treat = "0" if ar_treat=="N"
replace ar_treat = "" if ar_treat=="U"
destring ar_treat, replace
gen ar_bin_treat=(ar_bin==1 | ar_treat==1)
replace ar_bin_treat=. if ar_bin==. & ar_treat==.
saveold $hcvdir/temp0 ,replace
***Within 6 months***
use $hcvdir/temp0 ,clear
collapse (max) ar_bin_treat ar_bx ar_bx2 ar_bin, by(trr_id m6)
keep if m6==1
drop m6
rename ar_bin ar_bin_6m
rename ar_bx ar_bx_6m
rename ar_bx2 ar_bx2_6m
rename ar_bin_treat ar_bin_treat_6m
label var ar_bin_6m "AR in 6 months, binary"
label define ar_bin_6m 0 "N" 1 "Y"
label values ar_bin_6m ar_bin_6m
label var ar_bx_6m "AR by Bx in 6 months"
label define ar_bx_6m 0 "N" 1 "Y"
label values ar_bx_6m ar_bx_6m
label var ar_bx2_6m "AR by Bx 6 months(missing values filled in by ar_bin)"


label define ar_bx2_6m 0 "N" 1 "Y"
label values ar_bx2_6m ar_bx2_6m
label var ar_bin_treat_6m "AR clinically diagnosed or treated in 6 months"
label define ar_bin_treat_6m 0 "No" 1 "Yes"
label values ar_bin_treat_6m ar_bin_treat_6m
saveold $hcvdir/temp1 ,replace
***Within 1 year***
use $hcvdir/temp0 ,clear
collapse (max) ar_bin_treat ar_bx ar_bx2 ar_bin, by(trr_id within1yr)
keep if within1yr==1
drop within1yr
rename ar_bin ar_bin_1y
rename ar_bx ar_bx_1y
rename ar_bx2 ar_bx2_1y
rename ar_bin_treat ar_bin_treat_1y
label var ar_bin_1y "AR during 1st year, binary"
label define ar_bin_1y 0 "N" 1 "Y"
label values ar_bin_1y ar_bin_1y
label var ar_bx_1y "AR by Bx during 1st year"
label define ar_bx_1y 0 "N" 1 "Y"
label values ar_bx_1y ar_bx_1y
label var ar_bx2_1y "AR by Bx 1st year (missing values filled in by ar_bin0)"
label define ar_bx2_1y 0 "N" 1 "Y"
label values ar_bx2_1y ar_bx2_1y
label var ar_bin_treat_1y "AR clinically diagnosed or treated in 1st year"
label define ar_bin_treat_1y 0 "No" 1 "Yes"
label values ar_bin_treat_1y ar_bin_treat_1y
saveold $hcvdir/temp2 ,replace
***Within 2 years***
use $hcvdir/temp0 ,clear
collapse (max) ar_bin_treat ar_bx ar_bx2 ar_bin, by(trr_id m24)
keep if m24==1
drop m24
rename ar_bin ar_bin_2y
rename ar_bx ar_bx_2y
rename ar_bx2 ar_bx2_2y
rename ar_bin_treat ar_bin_treat_2y
label var ar_bin_2y "AR in 2 years, binary"
label define ar_bin_2y 0 "N" 1 "Y"
label values ar_bin_2y ar_bin_2y


label var ar_bx_2y "AR by Bx in 2 years"
label define ar_bx_2y 0 "N" 1 "Y"
label values ar_bx_2y ar_bx_2y
label var ar_bx2_2y "AR by Bx 2 yearss(missing values filled in by ar_bin)"
label define ar_bx2_2y 0 "N" 1 "Y"
label values ar_bx2_2y ar_bx2_2y
label var ar_bin_treat_2y "AR clinically diagnosed or treated in 2 years"
label define ar_bin_treat_2y 0 "No" 1 "Yes"
label values ar_bin_treat_2y ar_bin_treat_2y
saveold $hcvdir/temp3 ,replace
***Clinical AR outcome for the entire F/U period***
use $hcvdir/temp0 ,clear
collapse (max) ar_bin_treat ar_bx ar_bx2 ar_bin, by(trr_id)
rename ar_bin ar_bin_ever
label var ar_bin_ever "AR binary(ever)"
label define ar_bin_ever 0 "N" 1 "Y"
label values ar_bin_ever ar_bin_ever
rename ar_bx ar_bx_ever
label var ar_bx_ever "AR by Bx(ever)"
label define ar_bx_ever 0 "N" 1 "Y"
label values ar_bx_ever ar_bx_ever
rename ar_bx2 ar_bx2_ever
label var ar_bx2_ever "AR by Bx(ever,adjusted by ar_bin)"
label define ar_bx2_ever 0 "N" 1 "Y"
label values ar_bx2_ever ar_bx2_ever
rename ar_bin_treat ar_bin_treat_ever
label var ar_bin_treat_ever "AR ever diagnosed or treated"
label define ar_bin_treat_ever 0 "N" 1 "Y"
label values ar_bin_treat_ever ar_bin_treat_ever
saveold $hcvdir/temp4 ,replace
****Merging AR variables****
use $hcvdir/temp4 ,clear
merge 1:1 trr_id using $hcvdir/temp1 ,keep(master match) nogen
merge 1:1 trr_id using $hcvdir/temp2 ,keep(master match) nogen
merge 1:1 trr_id using $hcvdir/temp3 ,keep(master match) nogen
```

---

## 6. Racial category

```stata
// -----------------------------------
// Racial category- donor & recipient similar
// -----------------------------------

gen rec_race3=0
replace rec_race3=1*(can_race==16)+2*(can_race==2000)
replace rec_race3=. if missing(can_race)
label var rec_race3 "Recipient's race"
label define rec_race3 0 "Non-AA,non-Hispanic" 1 "AA" 2 "Hispanic"
label values rec_race3 rec_race3

gen can_race_simple=.
replace can_race_simple=1 if can_race==8
replace can_race_simple=2 if can_race==16
replace can_race_simple=3 if can_race==2000
replace can_race_simple=4 if !inlist(can_race,8,16,2000) & can_race!=.
label variable can_race_simple "Patient/s simplified race"
label define can_race_simple 1 "White" 2 "African American" 3 "Hispanic/Latino" 4 "Other/multi-racial"
label values can_race_simple can_race_simple
```

---

## 7. PRA (Panel Reactive Antibody), Time-varying PRA

```stata

// -----------------------------------
// PRA (Panel Reactive Antibody)
// -----------------------------------

****SRTR****
gen rec_peak_pra_con_srtr=can_last_srtr_peak_pra
label var rec_peak_pra_con_srtr "Peak PRA continuous, per SRTR"
*Peak PRA continuous,manually calculated*
egen rec_peak_pra_con_recode=rowmax(rec_pra_most_recent can_last_srtr_peak_pra can_last_alloc_pra can_la
label var rec_peak_pra_con_recode "Peak PRA continuous, manually recoded"
****Highly sensitized, 80% cut off****
gen rec_pra80=(rec_peak_pra_con_recode>80)
replace rec_pra80=. if missing(rec_peak_pra_con_recode)
label var rec_pra80 "Recipient recoded peak PRA>80%"
label define rec_pra80 0 "No" 1 "Yes"
label values rec_pra80 rec_pra80
```

---

```stata

// time varying PRA for candidate (maximum PRA within period) Use for candidate time
//categories used in match run (KAS) 0, 1-9, 10-19,20-29,30-39,40-49,...80-89,90-94,95,96,97,98,99,100
//convenient cutoffs binary at 80 (SRTR-way),
// -------------------------------------------------
// Time-varying PRA history for candidate time
// from pra_hist.dta
//
// before 2009: use max of !cPRA
// 2009 - 2010: use max of all 4 PRAs
// 2011 onward: use cPRA
//
// written by Xun Luo, commentary by Eric Chow 2016
// -------------------------------------------------
use "~/srtr1603/pra_hist.dta", clear
replace canhx_cpra=round(100*canhx_cpra) // make it out of 100
// keep certain variables (3 PRA variables, and cPRA)
keep px_id     canhx_begin_dt canhx_end_dt  ///
               canhx_srtr_peak_pra canhx_cur_pra canhx_alloc_pra ///
               canhx_cpra
// because we all enjoy order in our lives
sort px_id canhx_begin_dt
// check that histories are time-contiguous and not missing
assert canhx_begin_dt == canhx_end_dt[_n-1]+1 if px_id==px_id[_n-1]
assert !mi(canhx_begin_dt)
// create a history end-date (1603 censorship)
replace canhx_end_dt = min(canhx_end_dt,td(31aug2015))
// drop if the history begins after the censor date
13
drop if canhx_begin_dt > canhx_end_dt
sort px_id canhx_begin_dt // because entropy
// use max of all 4 PRA's if 2009-2010
gen pra_all_prahx = max(canhx_srtr_peak_pra,canhx_cur_pra,canhx_alloc_pra,canhx_cpra)
gen pra_09all_prahx = pra_all_prahx
// use max of 3 PRA's if 2008 and before
replace pra_09all_prahx=max(canhx_srtr_peak_pra,canhx_cur_pra,canhx_alloc_pra) if year(canhx_begin_dt)<2
// use cPRA if 2011 and onwards
gen pra_09all_11c_prahx=pra_09all_prahx
replace pra_09all_11c_prahx=canhx_cpra if year(canhx_begin_dt)>=2011
// the patient + history should be unique
isid px_id canhx_begin_dt
// keep the px_id, history range, and composite PRA
keep px_id canhx_begin_dt canhx_end_dt pra_09all_11c_prahx
// check contiguity again
assert canhx_begin_dt == canhx_end_dt[_n-1]+1 if px_id==px_id[_n-1]
assert 1==0
// save it to file
compress
saveold pra_hist_composite, v(12) replace
// fin
```

---


## 8. Kidney disease etiology/diagnosis

```stata

// -----------------------------------
// kidney disease etiology
// -----------------------------------
HTN, GN, DM, PKD, Other
gen dgn = 0 // other
// DM
replace dgn = 1 if inlist(can_dgn, 3011, 3012, 3038, 3039, 3069, 3070, 3071)
// HTN
replace dgn = 2 if can_dgn == 3040
// PKD
replace dgn = 3 if can_dgn == 3008
// FSGS
replace dgn = 4 if can_dgn == 3006
// GN
replace dgn = 5 if can_dgn == 3001 | can_dgn == 3041
// IGA
replace dgn = 6 if can_dgn == 3004
2.
gen rec_dx4=3
replace rec_dx4=0 if inlist(rec_dgn,3000,3001,3002,3003,3004,3005,3006,3016,3018,3024,3029,3031,3033,303
18
replace rec_dx4=1 if inlist(rec_dgn,3011,3012,3038,3039,3069,3070,3071)
replace rec_dx4=2 if rec_dgn==3040
replace rec_dx4=. if rec_dgn==.
label var rec_dx4 "Recipient's Dx,4 categories"
label define rec_dx4 0 "Glomerulonephritis" 1 "DMN" 2 "HTN" 3 "Everything else"
label values rec_dx4 rec_dx4

recode can_dgn (3011 3012 3039 3069 3070 3071 = 1 "dgn-DM") ///
    (3034 3040 3051 = 2 "dgn-htn") ///
    (3008 = 3 "dgn-PKD") ///
    (3050 3053 3055 3066  ///
    999 3007 3009 3010 3013 3014 3015 3017 3020 3021 3022 3023 3025 3026 ///
        3027 3028 3030 3032 3036 3044 3045 3046 3047 3048 3049 3052 3056 ///
        3057 3058 3059 3060 3063 = 4 "dgn-oth") ///
    (* = 0 "dgn-glomerular"), gen(can_dgn2)
drop can_dgn
rename can_dgn2 can_dgn

// Right now it's essentially the same thing as option 2 but this gives you convenience if you want t
/* Cause of ESRD */
gen cause=0
replace cause=1 if can_dgn==3000
replace cause=1 if can_dgn==3001
replace cause=1 if can_dgn==3002
replace cause=1 if can_dgn==3003
replace cause=1 if can_dgn==3004
replace cause=1 if can_dgn==3005
replace cause=1 if can_dgn==3006
replace cause=1 if can_dgn==3016
replace cause=1 if can_dgn==3018
replace cause=1 if can_dgn==3024
replace cause=1 if can_dgn==3029
replace cause=1 if can_dgn==3031
replace cause=1 if can_dgn==3033
replace cause=1 if can_dgn==3035
replace cause=1 if can_dgn==3041
replace cause=1 if can_dgn==3042
replace cause=1 if can_dgn==3043
replace cause=1 if can_dgn==3054
replace cause=1 if can_dgn==3067
replace cause=2 if can_dgn==3011
replace cause=2 if can_dgn==3012
replace cause=2 if can_dgn==3038
replace cause=2 if can_dgn==3039
replace cause=2 if can_dgn==3069
replace cause=2 if can_dgn==3070
replace cause=2 if can_dgn==3071
replace cause=3 if can_dgn==3008
replace cause=4 if can_dgn==3040
replace cause=5 if can_dgn==3019
replace cause=5 if can_dgn==3034
replace cause=5 if can_dgn==3050
replace cause=5 if can_dgn==3051
replace cause=5 if can_dgn==3053
replace cause=5 if can_dgn==3055


replace cause=6 if can_dgn==3010
replace cause=6 if can_dgn==3014
replace cause=6 if can_dgn==3015
replace cause=6 if can_dgn==3025
replace cause=6 if can_dgn==3028
replace cause=6 if can_dgn==3036
replace cause=6 if can_dgn==3052
replace cause=6 if can_dgn==3061
replace cause=7 if can_dgn==3007
replace cause=7 if can_dgn==3009
replace cause=7 if can_dgn==3013
replace cause=7 if can_dgn==3017
replace cause=7 if can_dgn==3026
replace cause=7 if can_dgn==3027
replace cause=7 if can_dgn==3030
replace cause=7 if can_dgn==3044
replace cause=7 if can_dgn==3045
replace cause=7 if can_dgn==3046
replace cause=7 if can_dgn==3047
replace cause=7 if can_dgn==3048
replace cause=7 if can_dgn==3049
replace cause=7 if can_dgn==3057
replace cause=7 if can_dgn==3059
replace cause=7 if can_dgn==3060
replace cause=7 if can_dgn==3063
replace cause=8 if can_dgn==3020
replace cause=8 if can_dgn==3021
replace cause=8 if can_dgn==3022
replace cause=8 if can_dgn==3023
replace cause=8 if can_dgn==3058
replace cause=9 if can_dgn==3037
replace cause=10 if !inlist(cause,1,2,4)
label define cause 1 "GN" 2 "DM" 4 "HTN" 10 "Others"
label values cause cause
```

---


## 9. Fix ABO/ABO mismatch

```stata

// -----------------------------------
// fix ABO
// -----------------------------------
// fix A1, A2 to A for bloodtype
foreach v of varlist don_abo can_abo {
    replace `v' = "AB" if inlist(`v', "A1B", "A2B")
    replace `v' = "A" if inlist(`v', "A1", "A2")
}
 encode can_abo,gen(cand_abo)
encode don_abo,gen(donor_abo) //makes them numeric with approp. value labels
```

---

```stata

// -------------------------------------------
// ABO mismatch
// -------------------------------------------
gen aboi=0
replace aboi=1 if (inlist(don_abo, "A", "A1", "A2", "B", "AB", "A1B", "A2B") & can_abo=="O") | ///
                  (inlist(don_abo, "B", "AB", "A1B", "A2B") & inlist(can_abo, "A", "A1", "A2")) | ///
                  (inlist(don_abo, "A", "A1", "A2", "AB", "A1B", "A2B") & can_abo=="B")
replace aboi=. if don_abo=="UNK" | can_abo=="UNK"
label var aboi "ABO incompatibility"
label define aboi 0 "N" 1 "Y"
label values aboi aboi
gen compatible = 0
    replace compatible = 1 if inlist(don_abo,"A") & inlist(can_abo,"A","A1","A1B","A2","A2B","AB")
    replace compatible = 1 if inlist(don_abo,"A1") & inlist(can_abo,"A","A1","A1B","A2","A2B","AB")
    replace compatible = 1 if inlist(don_abo,"A1B") & inlist(can_abo,"A1B","A2B","AB")
    replace compatible = 1 if inlist(don_abo,"A2") & inlist(can_abo,"A","A1","A1B","A2","A2B","AB")
    replace compatible = 1 if inlist(don_abo,"A2B") & inlist(can_abo,"A1B","A2B","AB")
    replace compatible = 1 if inlist(don_abo,"AB") & inlist(can_abo,"A1B","A2B","AB")
    replace compatible = 1 if inlist(don_abo,"B") & inlist(can_abo,"A1B","A2B","AB","B")
    replace compatible = 1 if inlist(don_abo,"O") & inlist(can_abo,"A","A1","A1B","A2","A2B","AB","O","B
    label var compatible "Compatible Groups"
//A2 incompatible, population & variable [pyc]
keep if inlist(can_abo, "O", "B")  // only those transplanted with O or B
// drop all the truly incompatible
tab don_abo can_abo, m
drop if can_abo=="O" & regexm(don_abo, "B")  // candidates O + donor with any B, incompatible
drop if regexm(don_abo, "A1")  // considered incompatible for both O and B candidates
drop if don_abo=="UNK"
drop if inlist(don_abo, "A", "AB")  // incompatible for both O and B candidates
tab don_abo can_abo, m
gen byte a2i = ///
    (can_abo=="O" & don_abo=="A2") | ///

(can_abo=="B" & inlist(don_abo, "A2", "A2B"))
    lab var a2i "A2 incompatibility indicator"
    capture lab drop a2i
    lab def a2i 0 "Compatible O & B recipient" 1 "A2 incompatible (A2 to O/B; or A2B to B)"
    lab val a2i a2i
// A2 incompatible
gen incompatible = 0
replace incompatible = 1 if inlist(don_abo,"A2") & inlist(can_abo,"B","O")
replace incompatible = 2 if inlist(don_abo,"A2B") & inlist(can_abo,"B")
label var incompatible "A2 Incompatible Groups"
label define incompatible 0 "Not" 1 "A2 to B or O recipient" 2 "A2B to B recipient"
label val incompatible incompatible
```

---


## 10. Age categories

```stata
// -----------------------------------
// Age category
// -----------------------------------
//Recipient Age Categories
    gen rec_age=0 if inrange(rec_age_at,18,44)
      replace rec_age=1 if inrange(rec_age_at,45,54)
      replace rec_age=2 if inrange(rec_age_at,55,64)
      replace rec_age=3 if inrange(rec_age_at,65,101)
        lab var rec_age "Recipient Age Categories"gen
        lab define RECAGE 0 "18-44" 1 "45-54" 2 "55-64" 3 "65+"
        lab values rec_age RECAGE
*For Below: Have to be careful of cut point if the original variable is truly continuous* [PC]
//Recipient Age Categories
gen rec_txage_cat=recode(rec_age_at_tx, 17, 34, 49, 64, 101)
        lab var rec_txage_cat "Recipient Age Categories"
    gen don_age_cat=recode(don_age, 17, 34, 49, 64, 101)
        lab var don_age_cat "Donor Age Categories"
```

---


## 11. Body Mass Index (BMI)

```stata
// -----------------------------------
// BMI
// -----------------------------------

gen rec_ht=rec_hgt_cm
replace rec_ht=can_hgt_cm if rec_ht==.
label var rec_ht "Recipient's ht in cm"
gen rec_wt=rec_wgt_kg
replace rec_wt=can_wgt_kg if rec_wt==.
label var rec_wt "Recipient's wt in kg"
*EDA showed that there were recipients with ht<50cm or wt<25kg, spurious*
replace rec_ht=. if rec_ht<50
replace rec_wt=. if rec_wt<25
gen rec_bmi2=rec_wt/((rec_ht/100)^2)
replace rec_bmi2=rec_bmi if rec_bmi2==.
drop rec_bmi
rename rec_bmi2 rec_bmi
label var rec_bmi "Recipient's BMI @ Tx"
**After data checking, decided to convert BMI<14 & BMI>50 to missing**
**CHECK WITH YOUR MENTOR in terms of how you define the "WEIRD" BMI value cut-off!!**
**See bmi_check.do (by Allan) for detail**
replace rec_bmi=. if rec_bmi<14 | (rec_bmi>50 & rec_bmi!=.)
3
*BMI categorical*
gen rec_bmi3=0*(rec_bmi<=25)+1*(rec_bmi>25 & rec_bmi<=30)+2*(rec_bmi>30)
replace rec_bmi3=. if rec_bmi==.
label var rec_bmi3 "Recipient's BMI, 3 categories"
label define rec_bmi3 0 "<=25" 1 "(25-30]" 2 ">30"
label values rec_bmi3 rec_bmi3
```

---


## 12. CIT (Cold Ischemia Time)

```stata
// -----------------------------------
// CIT (cold ischemia time/ cold ischemic time)
// -----------------------------------
*Continuous-per hour*
rename rec_cold_isch_tm rec_cit
replace rec_cit=. if rec_cit==99
label var rec_cit "CIT-continuous"
*Define categorical CIT based on distribution*
gen rec_cit3=0*(rec_cit<=12)+1*(rec_cit>12 & rec_cit<=24)+2*(rec_cit>24)
replace rec_cit3=. if missing(rec_cit)
label var rec_cit3 "CIT, 3 categories"
label define rec_cit3 0 "<=12hr" 1 "(12,24]hr" 2 ">24hr"
label values rec_cit3 rec_cit3
```

---


## 13. Donor Diabetes Mellitus (DM), Hypertension (HTN), terminal Serum Creatinine (SCr) >1.5, and manually recode Expanded Criteria Donor (ECD), SCr Assessment

```stata
// -------------------------------------------
// Donor DM, HTN, terminal SCr>1.5, and ECD recode manually
// -------------------------------------------
// definition of ECD: The expanded criteria donor (ECD) is any donor over the age of 60, or a donor over
// antiquated, allocation by KDPI now
// Note: Just use ECD! don_expand_don_ki. Compared with manual coding, consistent
*HTN- maximize information from don_hist_hyperten and don_htn*
tempvar don_htn1
gen `don_htn1'=0*(don_hist_hyperten==1)+1*(inrange(don_hist_hyperten,2,5))
replace `don_htn1'=. if inlist(don_hist_hyperten,998,.)
egen don_htn_bi=rowmax(`don_htn1' don_htn)
label var don_htn_bi "Donor HTN, binary"
label define don_htn_bi 0 "No" 1 "Yes"
label values don_htn_bi don_htn_bi
// ****Donor died from stroke****
gen don_stroke=0*(don_cod_don_stroke==0)+1*(don_cod_don_stroke==1)
replace don_stroke=. if don_cod_don_stroke==. | don_type!=0
label var don_stroke "Donor died from stroke"
label define don_stroke 0 "Died from other reasons" 1 "Died from stroke"
label values don_stroke don_stroke
24
*DM: binary*
gen don_dm=0*(don_hist_diab==1)+1*(don_hist_diab>=2 & don_hist_diab<=5)
replace don_dm=. if inlist(don_hist_diab,.,998)
label var don_dm "Donor DM-binary"
label define don_dm 0 "No" 1 "Yes"
label values don_dm don_dm
****ECD****
***Coded by SRTR***
gen don_ecd_srtr=0*(don_expand_don_ki==0)+1*(don_expand_don_ki==1)
replace don_ecd_srtr=. if don_type!=0
label var don_ecd_srtr "ECD per SRTR"
label define don_ecd_srtr 0 "SCD" 1 "ECD"
label values don_ecd_srtr don_ecd_srtr
****Manual recoding as per Port, Transplantation 2002****
gen don_ecd_recode=.
replace don_ecd_recode=1 if don_age>=60
replace don_ecd_recode=1 if inrange(don_age,50,59) & ( ///
                            (don_stroke==1 & don_high_creat==1) |  ///
                            (don_stroke==1 & don_htn_bi==1) | ///
                            (don_high_creat==1 & don_htn_bi==1) | ///
                            (don_stroke==1 & don_high_creat==1 & don_htn_bi==1) ///
                            )
replace don_ecd_recode=0 if don_age<50 | ///
                            (inrange(don_age,50,59) & ///
                            ((don_stroke==0 & don_high_creat==0) |  ///
                            (don_stroke==0 & don_htn_bi==0) | ///
                            (don_high_creat==0 & don_htn_bi==0) | ///
                            (don_stroke==0 & don_high_creat==0 & don_htn_bi==0)) ///
                            )
label var don_ecd_recode "ECD recoded by hand"
label define don_ecd_recode 0 "No" 1 "Yes"
label values don_ecd_recode don_ecd_recode
tab don_ecd_srtr don_ecd_recode
```

---

```stata
// -------------------------------------------
// Serum Creatinine ascertainment
// -------------------------------------------

// link to donor_deceased.dta
// ideally want to ascertain terminal serum creatinine of donor before donation

// tx_ki, if missing(don_creat), fill in from donor_deceased don_creat
// don_creat, don_max_creat

// untested: needs work!!!
gen serum_cr = don_creat
merge m:1 donor_id using donor_deceased, keepusing(don_creat? don_max etc..)
replace serum_cr = (don_creat? don_max etc..) if missing(serum_cr)

// donor scr variables from donor_deceased
don_creat don_peak_serum_creat don_final_serum_creat don_alloc_ecd_serum_creat don_max_creat
```


---

## 14. HLA mismatch

```stata
// -----------------------------------
// HLA mismatch
// -----------------------------------



// Some notes
rec_mm_equiv_tx is less complete than rec_mm_equiv_cur.
rec_mm_equiv_tx<=rec_mm_equiv_cur

//Continuous HLA is not a continuous concept

rename rec_mm_equiv_tx rec_hla_mis

*Binary-0 yes/no*
gen rec_hla0mis = (rec_hla_mis==0)
replace rec_hla0mis=. if rec_hla_mis==.
label var rec_hla0mis "HLA 100% matched"
label define rec_hla0mis 0 "HLA<100% matched" 1 "HLA100% matched"
label values rec_hla0mis rec_hla0mis

// HLA mismatch level (new SRTR way)
// ascertain A (0,1,2 mismatch or missing), AND DR (0,1,2 mismatch, or missing)
// DR and A is in graft survival, and A in pt-survival (SRTR models, 2015)
// untested code

gen hla_A = .
gen hla_DR = .

replace hla_A = 0 if rec_a_mm_equiv_tx == 0
replace hla_A = 1 if rec_a_mm_equiv_tx == 1
replace hla_A = 2 if rec_a_mm_equiv_tx == 2

replace hla_DR = 0 if rec_dr_mm_equiv_tx == 0
replace hla_DR = 1 if rec_dr_mm_equiv_tx == 1
replace hla_DR = 2 if rec_dr_mm_equiv_tx == 2


//HLA mismatch level (old SRTR way)
gen hla = .
 // 0 mismatch
replace hla=0 if rec_dr_mm_equiv_tx == 0 & rec_b_mm_equiv_tx == 0 & rec_a_mm_equiv_tx == 0
// A/B mismatch
replace hla=1 if rec_dr_mm_equiv_tx == 0 & mi(hla)
// one DR mismatch
replace hla=2 if rec_dr_mm_equiv_tx == 1
// two DR mismatch
replace hla=3 if (rec_dr_mm_equiv_tx == 2 | rec_a_mm_equiv_tx != 0 | rec_b_mm_equiv_tx != 0 | rec_dr_mm_equiv_tx != 0) & mi(hla)

assert !mi(hla)
```

---

## 15. Duration of pre-KT dialysis (vs listing?)

```stata
// -----------------------------------
// Duration of pre-KT dialysis
// -----------------------------------
*Preemptive*
gen rec_preemptive=0*(rec_pretx_dial=="Y")+1*(rec_pretx_dial=="N")
replace rec_preemptive=. if inlist(rec_pretx_dial,"U","")
label var rec_preemptive "Preemptive Tx"
label define rec_preemptive 0 "Tx after dialysis" 1 "Premptive Tx"
label values rec_preemptive rec_preemptive
.
*Continous (based on time)*
gen rec_dial_con=(rec_tx_dt-rec_dial_dt)/365.25
replace rec_dial_con=. if rec_dial_con<0
replace rec_dial_con=0 if rec_preemptive==1
// Some folks had rec_dial_con but preemptive=1. We need to account for that
replace rec_dial_con=. if rec_dial_con>rec_age
label var rec_dial_con "Time on dialysis pre-tx,in years"
*Categorical*

gen rec_dial4=0*(rec_premptive==1)+1*(rec_dial_con<=2)+2*(rec_dial_con>2 & rec_dial_con<=6)+3*(rec_dial_
replace rec_dial4=. if rec_dial_con==.
label var rec_dial4 "Time on dialysis pre-KTx,categorical"
label define rec_dial4 0 "Preemptive" 1  "<=2yr including preemptive" 2 "(2-6]" 3 ">6yr"
label values rec_dial4 rec_dial4
*******
//start with the transplant recipients
use E:\data\srtr1407\stata\kipa\tx_ki, clear
keep px_id pers_id org_ty rec_tx_dt *death*dt rec_age_at_tx can_diab can_abo can_gender can_race tfl_gra
append using E:\data\srtr1407\stata\kipa\tx_kp, keep(px_id pers_id org_ty rec_tx_dt *death*dt rec_age_at
drop if org_ty=="PA"
//years of prior dialysis / transplant
merge 1:1 px_id using E:\data\srtr1407\stata\kipa\cand, keepusing(can_dial_dt)
drop if _merge==2
drop _merge
gen origin_dt=rec_tx_dt
assert !mi(origin_dt)
gen date=origin_dt
gen kf = min(rec_tx_dt, rec_dial_dt, pers_esrd_first_dial_dt,can_dial_dt)
bys pers_id: egen kfail = min(kf)
gen years_rrt = origin_dt - kfail
replace years_rrt=0 if years_rrt<0 | mi(years_rrt)
//merge with transplant dataset based on px_id; obtain time to graft failure
preserve
tempfile years_kt
//correct failaure date from the two dataset
replace tfl_graft_dt=tfl_graft_dt_ki if mi(tfl_graft_dt)
replace pers_retx=pers_retxki if mi(pers_retx)
gen gfail_dt=min(tfl_graft_dt,rec_esrd_fail_dt,rec_esrd_grf_fail_dt,pers_retx,rec_fail_dt,rec_resum_main
keep pers_id rec_tx_dt gfail_dt
//graft failed before transplant date is regarded as 0 transplant day
replace gfail_dt = rec_tx_dt if gfail_dt < rec_tx_dt
//modify graft failure date by next transplant
sort pers_id rec_tx_dt
by pers_id: replace gfail_dt=min(gfail_dt,rec_tx_dt[_n+1])
by pers_id: gen tx_n=_n
sum tx_n
local n=r(max
reshape wide rec_tx_dt gfail_dt, i(pers_id) j(tx_n)
keep pers_id rec_tx_dt1-rec_tx_dt`n' gfail_dt1-gfail_dt`n'
duplicates drop
isid pers_id
save `years_kt', replace
restore
merge m:1 pers_id using `years_kt'
assert _merge==3
drop _merge
//accumulate previous transplant days
gen years_kt=0


forvalues j= 1/`n' {
                    quietly replace years_kt=years_kt+min(gfail_dt`j',origin_dt)-rec_tx_dt`j' if origin_
}
replace years_rrt=0 if mi(years_rrt) | years_rrt<0
assert years_rrt>=years_kt
isid px_id
keep px_id years_rrt years_kt
gen years_dial=years_rrt-years_kt

```

---

## 16. Hepatitis C Virus (HCV) (by diagnosis for candidate/by serostatus for recipient/by serostatus for donor)

```stata
// -----------------------------------
// Candidate HCV from Liver waitlist dgn
// -----------------------------------
//HCV among liver waitlist candidates
    gen hcv = cond(inlist(can_dgn, ///
    4104,4106,4204,4206,4216,4593),1,0) if !missing(can_dgn)
    replace hcv=1 if inlist(can_dgn2, ///
    4104,4106,4204,4206,4216,4593) & !missing(can_dgn2)
    replace hcv=1 if strpos(can_dgn_ost,"HCV")
    replace hcv=1 if strpos(can_dgn_ost,"TYPE C")
    replace hcv=1 if strpos(can_dgn_ost,"HEP C")
    replace hcv=1 if strpos(can_dgn_ost,"HEPATITIS C")
    replace hcv=1 if strpos(can_dgn_ost,"HEPATITIA C")
    replace hcv=1 if strpos(can_dgn_ost,"HAPETITIS C")
    replace hcv=1 if strpos(can_dgn_ost,"HEPATITISC")
        lab var hcv "HCV Cand. Diagnosis"
```

---

```stata
// -----------------------------------
// Recipient HCV
// -----------------------------------
This assumes rec_hcv_stat==. as HCV negative. Don't do this if HCV is an important variable in your stud
gen rec_hcv=rec_hcv_stat=="P" //rec_hcv_stat: recipient HCV serostatus
replace rec_hcv=1 if inlist(can_dgn, ///
    4104,4106,4204,4206,4216,4593),1,0) if !missing(can_dgn)
replace hcv=1 if inlist(can_dgn2, ///
    4104,4106,4204,4206,4216,4593) & !missing(can_dgn2)
2. HCV as exposure
    gen rec_hcv=cond(rec_hcv_stat=="P",1,cond(rec_hcv_stat=="N",0,.))
replace rec_hcv=1 if inlist(can_dgn, ///
        4104,4106,4204,4206,4216,4593),1,0) if !missing(can_dgn)
replace rec_hcv=1 if inlist(can_dgn2, ///
        4104,4106,4204,4206,4216,4593) & !missing(can_dgn2)
label var rec_hcv "Recipient HCV pos,binary"
label define rec_hcv 0 "HCV neg" 1 "HCV pos"
label values rec_hcv rec_hcv
```

---

```stata

// -----------------------------------
// Donor HCV
// -----------------------------------
1. This code assumes don_anti_hcv==. as HCV(-). Don't do this if HCV is an important variable in your st
gen don_hcv=don_anti_hcv=="P"
label var don_hcv "Donor HCV Ab"
label define don_hcv 0 "Neg/I" 1 "Pos"
label values don_hcv don_hcv
2. When Donor HCV is crucial part of analysis do not assume missing is HCV negative
global srtrdir="/dcs01/igm/segevlab/data/srtr1509/public"
local ddfile="$srtrdir/donor_deceased.dta"
use `ddfile',clear  //Separate file for live donor. Need to do some merging after this
gen don_hcv=.
replace don_hcv=1 if don_anti_hcv=="P"
4
replace don_hcv=0 if don_anti_hcv=="N"
label var don_hcv "Donor HCV Ab"
label define don_hcv 0 "Neg/I" 1 "Pos"
label values don_hcv don_hcv
```

---


## 17. Hepatocellular Carcinoma (HCC) (by diagnosis/by exception points)

```stata

// -----------------------------------
// HCC
// -----------------------------------
//HCC by exception point status
    capt drop hcc
    gen hcc = canhx_meld_diff_reason_cd ==8
    replace hcc= . if canhx_meld_diff==1 | canhx_meld_diff==2 ///
    | canhx_meld_diff==5 | canhx_meld_diff==12 | ///
    canhx_meld_diff==15 | canhx_meld_diff==16
//HCC by diagnosis
    gen hcc = cond(inlist(can_dgn,4400,4401),1,0) if !mi(can_dgn)
        replace hcc = 1 if inlist(can_dgn2, 4400,4401)
        replace hcc= 1 if strpos(can_dgn_ost, "HCC")
        replace hcc= 1 if strpos(can_dgn_ost, "HEPATOCELLULAR CARCINOMA")
            replace hcc= 1 if strpos(can_dgn_ost, "HEPATOCELLULARCARCINOMA")
```

---

## 18. Immunosuppression/Induction

```stata
// -----------------------------------
// Immunosuppression/ Induction
// induction - before transplant or on day, you get shots
// with goal to deactive T or B cells
// maintenance: start AFTER transplant that you carry on for life
// relevant to transplant outcomes
// includes substantial additions by Mary-Grace Bowring
// -----------------------------------



global srtrdir="/dcs01/igm/segevlab/data/srtr1509/public"
local isfile="$srtrdir/immuno.dta"


use `isfile',clear

****Induction****
tempvar is_induct00 is_induct11

gen `is_induct00'=(rec_drug_induction==1 & inlist(rec_drug_cd,16,-1,14,41,15,50))
gen `is_induct11'=(rec_drug_induction==1 & inlist(rec_drug_cd,42,43,52))

forvalues n=0/1 {

    replace `is_induct`n'`n''=. if rec_drug_induction==. | rec_drug_cd==.
	bys trr_id: egen is_induct`n'=max(`is_induct`n'`n'')

}


label var is_induct0 "1-Induction(depleting)"
label var is_induct1 "1-Induction(non-depleting)"
	 
			 
****Maintenance****
tempvar is_steroid_withdraw0
gen `is_steroid_withdraw0'=1
replace `is_steroid_withdraw0'=0 if rec_drug_maint==1 & inlist(rec_drug_cd,1,49)
replace `is_steroid_withdraw0'=. if missing(rec_drug_maint) | missing(rec_drug_cd)
bys trr_id: egen is_steroid_withdraw=min(`is_steroid_withdraw0')
				
collapse (max)  is_induct0 is_induct1 ///
                is_steroid_withdraw,by(trr_id) 
				
gen is_induct_other=(is_induct0==0 & is_induct1==0)
label var is_induct_other "1-Anything else besides depleting or non-depleting"			

tabstat is_induct0 is_induct1 is_induct_other ///
        is_steroid_withdraw,statistics(mean)
		
tab is_induct0 is_induct1,m

**************************************************************************************
** This do-file transforms the SRTR immunosuppression dataset so that it can be linked to tx_ki dataset using trr_id.

/** v3 update
- separated non-rabbit ATG from "thymo" as nratg
*/

/* Classification of the drugs:
1. Induction: OKT3, thymoglobulin (rabbit only), non-rabbit anti-thymocyte glubline (ALG, Atgam, NRATG/NRATS), Daclizumab, Basiliximab, Alemtuzumab, Rituximab.
2. Maintanence
	1) Steroids (Istedoid)
	2) CNI (Icni): Cyclosporin(including Neoral), Tacrolimus
	3) Anti-metabolites (Iantimeta): MMF(including Myfortic), mTOR(Sirolimus and Everolimus), Azathioprine  
3. Everything else is dumped. 	  */

local ver="1903"
global srtrdir "/dcs01/igm/segevlab/data/srtr`ver'/public"
global LOCAL /users/syu/016_dataset/001_steroid_rejection
	
set more off
use $srtrdir/immuno.dta, clear
cd "/users/syu/016_dataset/001_steroid_rejection"
	
// Generate indicators
gen Iokt3=inlist(rec_drug_cd, 16)
gen Ithymo=inlist(rec_drug_cd, 41)
gen Inratg=inlist(rec_drug_cd, -1 ,14, 15)
gen Idaclizumab=inlist(rec_drug_cd, 42)
gen Ibasiliximab=inlist(rec_drug_cd, 43)
gen Irituximab=inlist(rec_drug_cd, 52)
gen Ialemtuzumab=inlist(rec_drug_cd,50)
gen Isteroid=inlist(rec_drug_cd,1,2,49) /// WAW: I added in code 2 for methylpred as steroid induction. 
gen Icyclosporin=inlist(rec_drug_cd,3,4,-2,44,46,48)
gen Itacro=inlist(rec_drug_cd,5,53,54,59) ///	WAW: I added in code 59 for envarsus XR. 
gen Immf=inlist(rec_drug_cd,9,47,55,57)	/// WAW: I added in code 57 for generic myfortic. 
gen Imtor=inlist(rec_drug_cd,6,45)
gen Iaza=inlist(rec_drug_cd,8)

gen	Iinduction_steroid=inlist(rec_drug_cd,2)	
///	WAW: although high dose steroids (nearly always methlypred) are not considered induction per se, they are almost always given up front. 
///	WAW: for ESW work, this seems important to note as may be an mediator of the effects of later ESW. 

gen Iinduction=(Iokt3+ Ithymo+ Inratg+Idaclizumab+Ibasiliximab+ Irituximab+ Ialemtuzumab)>0
gen Icni=(Icyclosporin+Itacro)>0
gen Iantimeta=(Immf+Iaza)>0	/// WAW: I generally refer to aza/mmf only when I talk about antimetabolites, while mtor is an "antiproliferative". So i took mtor out of here.

// Create a list of trr_ids that has any information on this dataset
preserve
keep trr_id
duplicates drop
save imsup_all_v3, replace
restore

** But IS data for the most recent cohort is NOT comprehensive. Consider censoring everyone in the last year or something.

// See if they make clinical sense
drop if rec_drug_induction + rec_drug_maint + rec_drug_anti_rej ==0
drop if Iinduction & (rec_drug_induction + rec_drug_anti_rej) == 0

// Export
preserve
collapse (max) I* if rec_drug_induction, by(trr_id)
save imsup_ind, replace
restore

preserve
collapse (max) I* if rec_drug_maint, by(trr_id)
save imsup_maint, replace
restore

preserve
collapse (max) I* if rec_drug_anti_rej, by(trr_id)
save imsup_anti_rej, replace
restore

*** Also get the number of days for induction
preserve
keep if inlist(rec_drug_cd,16,-1,14,41,15,42,43,52,50)
keep if rec_drug_induction
replace rec_drug_cd=100 if rec_drug_cd==-1 //values need to be positive to reshape

keep trr_id rec_drug_cd rec_drug_days
reshape wide rec_drug_days, i(trr_id) j(rec_drug_cd)
mvencode _all, mv(0) override  //force the missing data to 0 

gen days_atg=rec_drug_days41
gen days_nratg=rec_drug_days100+rec_drug_days14+rec_drug_days15
gen days_il2=rec_drug_days42+rec_drug_days43
gen days_alm=rec_drug_days50
gen days_ritux=rec_drug_days52
gen days_okt3=rec_drug_days16

drop rec_drug_days*
compress
save imsup_days, replace
restore


foreach c in ind maint anti_rej {
	use imsup_`c', clear
	foreach v of varlist I* {
		rename `v' `v'_`c'
	}
	save imsup_`c', replace
}

// Lump all of them up into imsup_all_v3 dataset
use imsup_all_v3, clear
merge 1:1 trr_id using imsup_ind, nogen
merge 1:1 trr_id using imsup_maint, nogen
merge 1:1 trr_id using imsup_anti_rej, nogen
merge 1:1 trr_id using imsup_days, nogen

mvencode _all, mv(0) override
save "$LOCAL/imsup_all_`ver'", replace

// If _ever indicators are needed:  
/*
foreach drugname in Iokt3 Ithymo Inratg Idaclizumab Ibasiliximab Irituximab Ialemtuzumab Isteroid Icyclosporin Itacro Immf Imtor Iaza Iinduction Icni Iantimeta {
	egen `drugname'_ever=rowmax(`drugname'*)
}
save imsup_all_v3, replace
*/

```

---

## 19. Insurance

```stata

// -----------------------------------
// insurance
// -----------------------------------
gen insurance = 2 //"Other"
replace insurance = 1 if rec_primary_pay == 1 | rec_primary_pay == 8
// private (1) & self-pay (8)
replace insurance = 0 if (rec_primary_pay >=2 & rec_primary_pay <= 7) | rec_primary_pay == 13 | rec_prim
replace insurance=. if missing(rec_primary_pay)
label define insurance 0 "Public" 1 "Private" 2 "Other"
label values insurance insurance
```

---

## 20. Kidney Donor Risk Index (KDRI)/Kidney Donor Profile Index (KDPI)

```stata

// -----------------------------------
// KDRI/ KDPI
// -----------------------------------
//be mindful of time period when using this (can be odd to explore KDPI before it was adopted)
//this use the reference year 2013, ref year can be changed
//truncate serum creatinine and height at some extreme value and can replace htn with mean of previous y
<https://optn.transplant.hrsa.gov/ContentDocuments/Guide_to_Calculating_Interpreting_KDPI.pdf>

//AM: define path and reference year instead of hard-coding
global DATA /dcs01/igm/segevlab/data/srtr1903/public
global REF_Y 2018  //reference year for KDPI calculation
//keep recovered kidneys
use don_disposition donor_id don_org using $DATA/donor_disposition.dta,clear
keep if inlist(don_org,"LKI","RKI","EKI")
keep if inlist(don_disposition,5,6)
isid donor_id don_org
merge m:1 donor_id using $DATA/donor_deceased.dta,keep(match)
drop _merge
keep *id* *_dt* don_age *hgt* *wgt* *race*  ///
     don_hist_h* *diab* don_cad* *creat* *hcv* *hr* don_cad
duplicates drop
isid donor_id
//kdri
    gen age40 = don_age-40
    gen age18 = don_age-18
    replace age18 = 0 if don_age>=18
    gen age50 = don_age-50
    replace age50 = 0 if don_age<=50
    gen hgt170 = don_hgt_cm-170
    gen wgt80 = (don_wgt_kg-80)/5
    replace wgt80 = 0 if don_wgt_kg >=80
    gen aa = cond(don_race==16,1,0)
    gen hyp = cond(inrange(don_hist_hyp,2,5),1,0)
    gen diab = cond(inrange(don_hist_diab,2,5),1,0)
    gen cva = cond(don_cad_don==2,1,0)
    gen creat= don_creat
        replace creat= don_max_creat if creat==. & don_max_creat!=.
        replace creat=don_peak_ if creat==. & don_peak_!=.
        codebook creat
    //replace creat=8 if creat>8 & !mi(creat)
//SZ: creatinine should be capped at 8
    gen creat1 = creat-1
    gen creat1_5 = creat-1.5
    replace creat1_5 = 0 if creat <=1.5
    gen hcv=cond(don_anti_hcv=="P",1,0)


gen dcd = cond(don_non_hr_beat=="Y",1,0)
gen logkdri =
0.0128 * (age40)       ///
-0.0194 * (age18)      ///
+0.0107 * (age50)      ///
///
-0.0464 * (hgt170)/10  ///
-0.0199 * (wgt80)
///
+0.1790 * aa
///
+0.1260 * hyp
+0.1300 * diab
+0.0881 * cva
///
+0.2200 * (creat1)
-0.2090 * (creat1_5)
+0.2400 * (hcv)
+0.1330 * dcd
/// ///
/// /// ///
/// ///
///
gen y = year(don_recov_dt)
//AM: you don't really need to exponentiate
gen kdri = exp(logkdri)
sum kdri if y ==$REF_Y,det
local median r(p50)
gen kdri_med = kdri/`median'
hist kdri_med
drop if kdri==.
egen rank =rank(kdri_med) if y==$REF_Y
    sort y
    sort rank
bys y: gen total = _N if y==$REF_Y
sort rank
gen pctile=rank/total
gen kdpi = pctile*100
hist kdpi,freq
sort kdri
replace kdpi=kdpi[_n-1] if kdpi==.
gsort -kdri
replace kdpi=kdpi[_n-1] if kdpi==.
keep donor_id kdri kdpi logkdri don_recov_dt
capt drop _merge
save kdpi_id.dta,replace
```



---


## 21. Donor Risk Index (DRI)


```stata
// -----------------------------------
// DRI
// -----------------------------------
//you can exclude rec_tx_procedure_ty/sharing/CIT to calculate "adjusted DRI" (pre-tx DRI)
gen log_dri = ///
0.154*(inrange(don_age, 40, 49)) + ///
0.274*(inrange(don_age, 50, 59)) + ///
0.424*(inrange(don_age, 60, 69)) + ///
0.501*(inrange(don_age, 70, 99)) + ///
0.079*(don_cad_don_cod==1) + ///
0.145*(don_cad_don_cod==2) + ///
0.184*(!inlist(don_cad_don_cod, 1, 2, 3)) + ///  // not Anoxia (1), CVA/stroke (2) , or trauma (3)
0.176*(don_race==16) + ///
    0.126*(!inlist(don_race, 8, 16)) + ///
    0.411*(don_non_hr_beat=="Y") + ///
    0.066*((170-don_hgt_cm)/10) +  ///
    0.422*(rec_tx_procedure_ty != 701) + ///
    0.105*(regional_share) + ///
    0.244*(national_share)+ ///
    0.010*(rec_cold_isch_tm)
gen don_dri = exp(log_dri)
```

---


## 22. Estimated Post Transplant Survival (EPTS)

```stata

// -----------------------------------
// EPTS
// -----------------------------------
//see https://optn.transplant.hrsa.gov/media/1511/guide_to_calculating_interpreting_epts.pdf

gen epts_age25 = max(rec_age_at-25, 0)
//NOTE: not clear how to ascertain post-listing incident DM
gen rec_dm = inrange(can_diab_ty, 2, 5) if inrange(can_diab_ty, 1, 5)
replace rec_dm = inlist(can_diab, 2, 3) if can_diab<=3 & missing(rec_dm)
//NOTE: this is for EPTS as of date of transplant
gen rrt_y = (rec_tx_dt-rec_dial_dt)/365.25
gen byte rec_prev_tx = (rec_prev_ki==1 | rec_prev_li==1 | rec_prev_hr==1 ///
    | rec_prev_lu==1 | rec_prev_pa==1)

gen raw_epts = ///
    0.047*epts_age25 + ///
   -0.015*rec_dm*epts_age25 + ///
    0.398*rec_prev_tx + ///
   -0.237*rec_dm*rec_prev_tx + ///
    0.315*log(rrt_y+1) + ///
    0.130*(rrt_y==0) + ///
   -0.348*rec_dm*(rrt_y==0) + ///
    1.262*rec_dm
sum raw_epts, detail
count if missing(raw_epts)
assert r(N)<100
local N_EPTS = _N-r(N)
sort raw_epts
gen rec_epts = _n/`N_EPTS' if !missing(raw_epts)
```

---


## 23. Machine perfusion

```stata

// -----------------------------------
// Machine perfusion
// -----------------------------------
use $srtrdir/donor_disposition.dta, clear
keep donor_id px_id don_org
drop if px_id==.
keep if inlist(don_org,"LKI","RKI","EKI")  // drop all other organs (especially pancrease (PA))
isid px_id               // no news is good news here.
tempfile donor
saveold `donor'
use $hcvdir/hcv_ar,clear
capture drop organ
merge 1:1 px_id using `donor',keep(master match) nogen
saveold $hcvdir/hcv_ar ,replace
gen rec_which_k=0*(don_org=="LKI")+1*(don_org=="RKI")+2*(don_org=="EKI")
label var rec_which_k "Which kidney Tx"
label define rec_which_k 0 "Lt" 1 "Rt" 2 "En bloc"
label values rec_which_k rec_which_k
gen rec_pump=cond(don_lf_ki_pump=="Y",1,0,.) if rec_which_k==0
replace rec_pump=cond(don_rt_ki_pump=="Y",1,0,.) if rec_which_k==1
replace rec_pump=1 if (don_rt_ki_pump=="Y" | don_lf_ki_pump=="Y") & rec_which_k==2
replace rec_pump=. if don_rt_ki_pump=="" & don_lf_ki_pump=="" & rec_pump!=.
****Reconcile with the SRTR machine perfusion variable****
8
replace rec_pump=1 if rec_received_on_pump==1 & missing(rec_pump)
label var rec_pump "Donor kidney pumped"
label define rec_pump 0 "Cold storage" 1 "Pumped"
label values rec_pump rec_pump
```

---


## 24. Model for End-Stage Liver Disease (MELD) (listing /transplant)

```stata
// -----------------------------------
// MELD: INR, Serum CR, bilirubin, dialysis within past week
// -----------------------------------
//MELD AT TX (BIOLOGIC)
  gen byte meld=can_last_srtr_lab_meld - 6200 ///
        if inrange(can_last_srtr_lab_meld, 6206, 6240)
  replace meld=41 if inlist(can_last_srtr_lab_meld, 3010, 6011, 6012) //*urgent*
  replace meld=6 if can_last_srtr_lab_meld ==3020 //"non-urgent*
//MELD AT LISTING (BIOLOGIC)
  gen byte meld_listing=can_init_srtr_lab_meld - 6200 ///
            if inrange(can_init_srtr_lab_meld,6206, 6240)

//anoxia
// CVA/stroke
replace meld_listing=41 if inlist(can_init_srtr_lab_meld, 3010, 6011, 6012) //*urgent*
  replace meld_listing=6 if can_init_srtr_lab_meld ==3020 //"non-urgent*
*MELD AT TX (ALLOCATION)
// status 1A,1B treated as 41
  gen byte meld=can_last_stat - 6200 ///
        if inrange(can_last_stat,6206, 6240)
  replace meld=41 if inlist(can_last_stat, 3010, 6011, 6012) //*urgent*
  replace meld=6 if can_last_stat ==3020 //"non-urgent*
  tab can_last_stat if missing(meld) //make sure these are all inactive
// if inactive (6999) but transplanted (?) or you want to include inactive time on waitlist then replace
// There is an OPTN version! sensitivity analysis
//MELD AT LISTING (ALLOCATION)
  gen byte meld_listing=can_init_stat - 6200 ///
            if inrange(can_init_stat,6206, 6240)
  replace meld_listing=41 if inlist(can_init_stat, 3010, 6011, 6012) //*urgent*
  replace meld_listing=6 if can_init_stat ==3020 //"non-urgent*
  tab can_init_stat if missing(meld_listing) //make sure these are all inactive
//time-varying MELD AT LISTING (ALLOCATION) (stathist.dta)
    gen byte ameld=canhx_stat_cd - 6200 ///
    if inrange(canhx_stat_cd,6206, 6240)
    replace ameld=41 if inlist(canhx_stat_cd, 3010, 6011, 6010, 6012) //*urgent*
    replace ameld=6 if canhx_stat_cd ==3020 //"non-urgent*
    replace ameld = 40 if canhx_stat_cd>6240 & canhx_stat_cd<=6282
    replace ameld=99 if canhx_stat_cd ==6999
    replace ameld = 6 if canhx_stat_cd<6206 & canhx_stat_cd>=6188
```

---


## 25. Previous solid organ transplant (Tx)/ previous kidney transplant (KT)

```stata
// -------------------------------------------
// Previous solid organ Tx or previous KT-> just to show where to find it
// -------------------------------------------
*****Repeat KT variable based on SRTR can_prev_tx variable*****
rename can_prev_ki rec_repeat_kt
label var rec_repeat_kt "Repeat KT based on SRTR can_prev_tx variable"
****Previous organ transplantation****
rename can_prev_tx rec_prev_tx
label var rec_prev_tx "Previous solid organ Tx"
label define rec_prev_tx 0 "N" 1 "Y"
label values rec_prev_tx rec_prev_tx
```

---


## 26. Organ Sharing

```stata
// -----------------------------------
// Organ Sharing
// need to merge with srtr1407/../institutions.dta
// -----------------------------------
// untested code
gen local = 0
gen regional = 0
gen national = 0
// must merge to institutions.dta
// to get rec_region, don_region
// other race (not white or black)
replace local = 1 if !(don_org_shared)
replace regional = 1 if (don_org_shared == 1) & (rec_region == don_region)
replace national = 1 if !(local) & !(regional)
```

---


## 27. De-identification Code

```stata
*DE-IDENTIFICATION CODE:

/* run this code on a dataset to remove identifiers, dates (less than year), geographic identifiers, or
// -------------------------------------------------
// SRTR de-identification     v1.0      Sep 14, 2016
// CAUTION: this code not guaranteed to de-identify
// data, please review with your analyst.
// * drop identifying variables *
foreach keyw in    id dt date day mrn name ssn  ///
                   address city county fips zip ///
                   tel fax email                ///
        { capture drop *`keyw'* }
// * drop date-format variables *
ds, has(format %d* %-d* %t* %-t*)
capture drop `r(varlist)'
// * no one older than 90 *
foreach var of varlist *age* {
    assert `var' < 90 }
// -------------------------------------------------
```

---


## 28. Transplant Date

```stata
// -------------------------------------------
// transplant date
// -------------------------------------------
//If you only deal with Tx recipients but not candidates, just think about tx_dt

//for waitlist removals and transplant rates when using candidate.dta sets
gen tx_dt= rec_tx_dt
replace tx_dt =can_rem_dt if inlist(can_rem_cd,4)
//if estimating DD transplant rates
replace tx_dt =can_rem_dt if inlist(can_rem_cd,4,18,19,21,22)
replace tx_dt = . if inlist(can_rem_cd,14,15)
//to censor live donors
replace tx_dt = . if can_rem_cd==15
```

---


## 29. Living Donor Programmed Kidney Injury (LDPKI)

```stata
*LKDPI or LD KDPI

// -------------------------------------------
// Calculate LKDPI
// -------------------------------------------
// m2m?
gen m2m = 0
replace m2m = 1 if (don_fem == 0 & can_fem == 0)
// MIN of DRWR and 0.9
gen drwr = don_wgt_kg / rec_wgt_kg
replace drwr = 0.9 if drwr > 0.9 & !missing(drwr)
sum don_bp_preop_syst, detail
gen sbp = don_bp_preop_syst
tab don_hist_cigarette

gen smok = don_hist_cigarette == "Y"
// are they ABO incompatible?
tab don_abo can_abo
capture drop aboi
gen aboi = .
replace aboi = inlist(don_abo, "A", "A1", "A2") & inlist(can_abo, "B", "O") | ///
               inlist(don_abo, "A1B", "A2B", "AB") & inlist(can_abo, "A", "A1","A2","B","O") | ///
               inlist(don_abo, "B") & inlist(can_abo, "A", "A1", "A2","O")
tab aboi
capture drop lkdpi
// calculate lkdpi
gen lkdpi = -11.3 ///
    + 1.85 *  dage50_spline ///
    - 0.381 * ld_egfr_preop ///
    + 1.17 *  don_bmi ///
    + 22.34 * don_black ///
    + 14.33 * smok ///
    + 0.44 *  sbp ///
    - 21.68 * m2m ///
    + 27.30 * aboi ///fol_age
    - 10.61 * !don_related ///
    + 8.57  * rec_b_mm_equiv_tx ///
    + 8.26 *  rec_dr_mm_equiv_tx ///
    -50.87 *  drwr
sum lkdpi, detail
//// LDKT, version 2 (Based on the above version; defines all variables needed.)
global srtrdir "~/foo/bar"
use $srtrdir/tx_ki, clear
keep if don_ty=="L"
merge 1:1 donor_id using $srtrdir/donor_live, keep(master match)
// Age spline
gen dage50_spline = (don_age-50)*(don_age>50)
// Sex
gen don_female = don_gender=="F"
gen can_female = can_gender=="F"
// Race
gen don_black = don_race==16
// CKD epi equation for eGFR
gen ld_male = don_gender=="M"
30

gen k = cond(ld_male, 0.9, 0.7)
gen a = cond(ld_male, -0.411, -0.329)
gen egfr = 141 * ///
    (min(don_ki_creat_preop/k, 1))^a * ///
    (max(don_ki_creat_preop/k, 1))^(-1.209) * ///
    0.993^fol_age * ///
    cond(ld_male, 1, 1.018) * ///
    cond(don_black, 1.159, 1) ///
    if !missing(don_ki_creat_preop)
sum egfr, detail
rename egfr ld_egfr_preop
// Donor BMI (20191017: why can_wgt_kg???)
gen don_bmi = can_wgt_kg/(don_hgt_cm/100)^2
// donor related
gen don_related = inlist(don_relationship_ty, 1,2,3,4,5,6)
// m2m?
gen m2m = 0
replace m2m = 1 if (don_female == 0 & can_female == 0)
// MIN of DRWR and 0.9
gen drwr = don_wgt_kg / rec_wgt_kg
replace drwr = 0.9 if drwr > 0.9 & !missing(drwr)
sum don_bp_preop_syst, detail
gen sbp = don_bp_preop_syst
tab don_hist_cigarette
gen smok = don_hist_cigarette == "Y"
// are they ABO incompatible?
tab don_abo can_abo
capture drop aboi
gen aboi = .
replace aboi = inlist(don_abo, "A", "A1", "A2") & inlist(can_abo, "B", "O") | inlist(don_abo, "A1B", "A2
tab aboi
capture drop lkdpi
// calculate lkdpi
gen lkdpi = -11.3 + 1.85 *  dage50_spline - 0.381 * ld_egfr_preop + 1.17 *  don_bmi + 22.34 * don_black
sum lkdpi, detail
```

---

